<a href="https://colab.research.google.com/github/maciekpiotrowski/dw_matrix_road_sign/blob/master/road_signs_day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
cd /content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [25]:
import pandas as pd
import numpy as np
import os
import datetime
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure
from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [7]:
!pip install hyperopt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')
X_train, y_train = train['features'],train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim ==1: y_train=to_categorical(y_train)
if y_test.ndim ==1: y_test=to_categorical(y_test)

In [0]:
y_test.shape

(4410, 43)

In [0]:
input_shape=X_train.shape[1:]
num_classes=y_train.shape[1]

In [0]:
def train_model(model,X_train,y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
  logdir=os.path.join("logs",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback=tf.keras.callbacks.TensorBoard(logdir,histogram_freq=1)
  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size',128),
      epochs=params_fit.get('epochs',5),
      verbose=params_fit.get('verbose',1),
      validation_data=params_fit.get('validation_data',(X_train,y_train)),
      callbacks=[tensorboard_callback]
  )
  return model



In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm=np.argmax(y_test,axis=1)
  y_pred_prob=model_trained.predict(X_test)
  y_pred=np.argmax(y_pred_prob,axis=1)
  return scoring(y_test_norm,y_pred)


In [0]:
def train_and_predict(model):
  model_trained=train_model(model,X_train,y_train)
  return predict(model_trained,X_test, y_test)

In [0]:
def get_cnn_v5(input_shape,num_classes):
   return Sequential([
                      
  Conv2D(filters=32, kernel_size=(3,3),activation='relu',input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3),activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3),activation='relu',padding='same'),
  Conv2D(filters=64, kernel_size=(3,3),activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3),activation='relu',padding='same'),
  Conv2D(filters=64, kernel_size=(3,3),activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),
  Dense(1024,activation='relu'),
  Dropout(0.3),

  Dense(1024,activation='relu'),
  Dropout(0.3),
  
  Dense(num_classes,activation='softmax')  
   ])
#get_cnn_v4(input_shape,num_classes).summary()
train_and_predict(get_cnn_v5(input_shape,num_classes))

In [0]:
df=pd.read_csv('data/signnames.csv')
labels_dict=df.to_dict()['b']

In [17]:
model=get_cnn_v5(input_shape,num_classes)
model_trained=train_model(model,X_train,y_train)
predict(model_trained,X_test,y_test)

Epoch 1/5
272/272 [==============================] - 4s 15ms/step - loss: 2.4572 - accuracy: 0.2989 - val_loss: 1.1651 - val_accuracy: 0.6241
Epoch 2/5
272/272 [==============================] - 4s 13ms/step - loss: 0.9024 - accuracy: 0.6999 - val_loss: 0.2739 - val_accuracy: 0.9259
Epoch 3/5
272/272 [==============================] - 4s 13ms/step - loss: 0.4052 - accuracy: 0.8726 - val_loss: 0.0961 - val_accuracy: 0.9701
Epoch 4/5
272/272 [==============================] - 4s 13ms/step - loss: 0.2541 - accuracy: 0.9241 - val_loss: 0.0543 - val_accuracy: 0.9822
Epoch 5/5
272/272 [==============================] - 4s 13ms/step - loss: 0.1807 - accuracy: 0.9468 - val_loss: 0.0335 - val_accuracy: 0.9902


0.9777777777777777

In [57]:
model_trained.evaluate(X_test,y_test)[1]

138/138 [==============================] - 0s 3ms/step - loss: 0.0883 - accuracy: 0.9778


0.9777777791023254

In [0]:
def get_model(params):
   return Sequential([
                      
  Conv2D(filters=32, kernel_size=(3,3),activation='relu',input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3),activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_one']),

  Conv2D(filters=64, kernel_size=(3,3),activation='relu',padding='same'),
  Conv2D(filters=64, kernel_size=(3,3),activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_two']),

  Conv2D(filters=128, kernel_size=(3,3),activation='relu',padding='same'),
  Conv2D(filters=128, kernel_size=(3,3),activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_three']),

  Flatten(),
  Dense(1024,activation='relu'),
  Dropout(params['dropout_dense_block_one']),

  Dense(1024,activation='relu'),
  Dropout(params['dropout_dense_block_two']),
  
  Dense(num_classes,activation='softmax')  
   ])

In [0]:
def func_obj(params):
  model=get_model(params)
  model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size',128)),
      epochs=5,
      verbose=0
  )
  accuracy=model.evaluate(X_test,y_test)[1]
  print(params,'accuracy={}'.format(accuracy))
  return {'loss':-accuracy,'status':STATUS_OK,'model':model}

In [60]:
space={
    'batch_size':hp.quniform('batch_size',100,200,10),
    'dropout_cnn_block_two':hp.uniform('dropout_cnn_block_two',0.3,1),
    'dropout_cnn_block_one':hp.uniform('dropout_cnn_block_one',0.3,1),
    'dropout_cnn_block_three':hp.uniform('dropout_cnn_block_three',0.3,1),
    'dropout_dense_block_one':hp.uniform('dropout_dense_block_one',0,0.7),
    'dropout_dense_block_two':hp.uniform('dropout_dense_block_two',0,0.7)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    25,
    Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 2.9534 - accuracy: 0.0625

 16/138 [==>...........................]
 - ETA: 0s - loss: 1.0369 - accuracy: 0.6895

 26/138 [====>.........................]
 - ETA: 0s - loss: 1.3027 - accuracy: 0.6971

 40/138 [=======>......................]
 - ETA: 0s - loss: 1.0202 - accuracy: 0.7719

 51/138 [==========>...................]
 - ETA: 0s - loss: 1.0958 - accuracy: 0.7108

 65/138 [=============>................]
 - ETA: 0s - loss: 0.9759 - accuracy: 0.7409


In [1]:
best

NameError: ignored